In [1]:
import pandas as pd


In [43]:
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np

### 개인 정보: 성별, 연령대, 국가, 유입경로
### 고객의 개인정보와 2021-1월-14일 전후의 가입자수의 상관관계 구하기

In [33]:
look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker.csv')

C:\Users\LG\AppData\Local\Temp\ipykernel_5856\851929299.py:1: DtypeWarning: Columns (2,3,4,5,8,9,12,14,15,18,19,20,21,22,23,24,31) have mixed types. Specify dtype option on import or set low_memory=False.
  look = pd.read_csv('C:/Users/LG/Desktop/데이터분석/프로젝트/최종프로젝트/3차/looker.csv')


In [34]:
# 형변환
def convert_type(df):
    for col in df.columns:
        if '_at' in col:    # 시간 데이터
            df = df.assign(**{col:pd.to_datetime(df[col])})
            print('[+] {0:<30} >>> datetime'.format(col))
        elif '_id' in col:  # id 컬럼 -> obj형으로
            df = df.assign(**{col:df[col].astype('object')})
            print('[+] {0:<30} >>> object'.format(col))
    return df

In [35]:
look = convert_type(look)

[+] user_id                        >>> object
[+] account_created_at             >>> datetime
[+] order_id                       >>> object
[+] order_items_id                 >>> object
[+] order_created_at               >>> datetime
[+] session_id                     >>> object
[+] login_user_id                  >>> object
[+] sess_created_at                >>> datetime
[+] inventory_item_id              >>> object
[+] product_id                     >>> object
[+] inventory_created_at           >>> datetime
[+] inventory_sold_at              >>> datetime


In [36]:
pd.set_option('display.max_columns', None)

In [ ]:
df = look.copy()


In [144]:
df.describe(include='all')

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,order_id,order_items_id,status,sale_price,order_created_at,session_id,sequence_number,login_user_id,sess_created_at,sess_traffic_source,uri,event_type,ip_address,sess_state,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost
count,112960.0,112960.000000,112960,112960,111840,112960,112960.000000,112960.000000,112960,112960,93004.0,93004.0,93004,93004.000000,93004,2343260,2.343260e+06,1217589.0,2343260,2343260,2343260,2343260,2343260,2343260,2320938,501300.0,501300.0,501300,500892,501271,501300,181759,501300.000000
unique,75608.0,NaN,2,227,7483,16,NaN,NaN,5,NaN,73703.0,82409.0,3,NaN,NaN,681759,NaN,80044.0,NaN,5,35530,6,681683,231,8775,490705.0,29046.0,26,2752,27236,NaN,NaN,NaN
top,83768.0,NaN,M,Guangdong,Shanghai,China,NaN,NaN,Search,NaN,102915.0,161807.0,Shipped,NaN,NaN,8ce484ca-2e5f-42e3-b092-550ee21ed9f1,NaN,32996.0,NaN,Email,/cart,product,150.27.245.192,Guangdong,Shanghai,235949.0,18795.0,Intimates,Allegra K,Wrangler Men's Premium Performance Cowboy Cut ...,NaN,NaN,NaN
freq,25.0,NaN,56511,6010,2852,38806,NaN,NaN,79143,NaN,16.0,5.0,42893,NaN,NaN,16,NaN,161.0,NaN,1052132,595994,845607,16,125025,58310,5.0,61.0,37118,17383,163,NaN,NaN,NaN
mean,NaN,41.106188,NaN,NaN,NaN,NaN,28.381581,25.001698,NaN,2021-08-14 19:33:27.280453120,NaN,NaN,NaN,59.853164,2022-11-01 17:14:38.981549568,NaN,3.150051e+00,NaN,2022-03-11 17:38:32.366190336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-12 11:43:22.254936832,2022-10-24 07:44:52.389372672,28.664935
min,NaN,12.000000,NaN,NaN,NaN,NaN,-43.007536,-158.164931,NaN,2019-01-02 00:00:00,NaN,NaN,NaN,0.020000,2019-01-06 00:00:00,NaN,1.000000e+00,NaN,2019-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-08 00:00:00,2019-01-06 00:00:00,0.008300
25%,NaN,26.000000,NaN,NaN,NaN,NaN,26.269040,-51.078787,NaN,2020-04-24 00:00:00,NaN,NaN,NaN,24.650000,2022-03-09 00:00:00,NaN,1.000000e+00,NaN,2021-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-04-14 00:00:00,2022-02-24 00:00:00,11.370000
50%,NaN,41.000000,NaN,NaN,NaN,NaN,35.202550,4.851624,NaN,2021-08-18 00:00:00,NaN,NaN,NaN,39.990002,2023-03-02 00:00:00,NaN,2.000000e+00,NaN,2022-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-05 00:00:00,2023-02-21 00:00:00,19.842060
75%,NaN,56.000000,NaN,NaN,NaN,NaN,40.725810,116.372171,NaN,2022-12-06 00:00:00,NaN,NaN,NaN,69.949997,2023-10-06 00:00:00,NaN,4.000000e+00,NaN,2023-06-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-11 00:00:00,2023-09-27 00:00:00,34.560000
max,NaN,70.000000,NaN,NaN,NaN,NaN,64.865194,153.560238,NaN,2024-01-16 00:00:00,NaN,NaN,NaN,999.000000,2024-01-20 00:00:00,NaN,1.300000e+01,NaN,2024-01-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-21 00:00:00,2024-01-21 00:00:00,557.151002


- cut 함수 기본 사용법
- 기능: 연속적인 데이터 값을 지정된 구간으로 나누어 범주화합니다.
- 주요 파라미터:
- x: 나누고자 하는 데이터 배열.
- bins: 구간의 경계값이나 구간의 수를 지정.
- labels: 구간별로 지정할 라벨. 지정하지 않으면 구간의 범위가 라벨로 사용됩니다.
- right: 구간의 오른쪽을 포함할지 여부. 기본값은 True.


In [145]:
bins = [10,19,29,39,49,59,69,79]
bins_names = [10,20,30,40,50,60,70]


In [146]:
df['age2'] = pd.cut(df['age'], bins, labels = bins_names)

In [147]:
df['age2'].value_counts()

age2
60    19491
50    19357
20    19131
40    18963
30    18889
10    15252
70     1877
Name: count, dtype: int64

In [148]:
df[df['age']==70][['age','age2']]

,age,age2
209,70.0,70
1153,70.0,70
1385,70.0,70
1386,70.0,70
2098,70.0,70
...,...,...
2759747,70.0,70
2760948,70.0,70
2762632,70.0,70
2766870,70.0,70


In [149]:
# 실험군: 2024 신규 유입 유저
treatment = df[df.account_created_at >= '2024-01-11'].copy()

In [150]:
# 대조군: 기존 유저
control = df[df.account_created_at < '2024-01-11'].copy()

In [151]:
print(
    f'전체 유저 수 : {df.user_id.nunique(dropna=True)}명\n\
실험군 (treatment) : {treatment.user_id.nunique()}명\n\
대조군 (control) : {control.user_id.nunique()}명'
    )

전체 유저 수 : 75608명
실험군 (treatment) : 2108명
대조군 (control) : 73500명


In [185]:
# 실험군의 연령별 가입자수
test_10 = treatment[treatment['age2']==10]['user_id'].nunique()
test_20 = treatment[treatment['age2']==20]['user_id'].nunique()
test_30 = treatment[treatment['age2']==30]['user_id'].nunique()
test_40 = treatment[treatment['age2']==40]['user_id'].nunique()
test_50 = treatment[treatment['age2']==50]['user_id'].nunique()
test_60 = treatment[treatment['age2']==60]['user_id'].nunique()
test_70 = treatment[treatment['age2']==70]['user_id'].nunique()

In [186]:
# 대조군의 연령별 가입자수
cont_10 = control[control['age2']==10]['user_id'].nunique()
cont_20 = control[control['age2']==20]['user_id'].nunique()
cont_30 = control[control['age2']==30]['user_id'].nunique()
cont_40 = control[control['age2']==40]['user_id'].nunique()
cont_50 = control[control['age2']==50]['user_id'].nunique()
cont_60 = control[control['age2']==60]['user_id'].nunique()
cont_70 = control[control['age2']==70]['user_id'].nunique()

In [187]:
## Part1. Chi-square Test Report
age10 = [test_10, cont_10] # A와 B의 10대 유저 수
age20 = [test_20, cont_20] # A와 B의 20대 유저 수
age30 = [test_30, cont_30] # A와 B의 30대 유저 수
age40 = [test_40, cont_40] # A와 B의 40대 유저 수
age50 = [test_50, cont_50] # A와 B의 50대 유저 수
age60 = [test_60, cont_60] # A와 B의 60대 유저 수
age70 = [test_70, cont_70] # A와 B의 70대 유저 수

In [188]:
## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([age10,age20,age30,age40,age50,age60,age70], columns=['treatment', 'control'], index=["10대",'20대',"30대",'40대',"50대",'60대','70대'])

In [189]:
cont_table

,treatment,control
10대,284,9898
20대,361,12492
30대,306,12281
40대,337,12417
50대,409,12581
60대,372,12561
70대,39,1270


In [190]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([age10,age20,age30,age40,age50,age60,age70])

In [191]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["10대",'20대',"30대",'40대',"50대",'60대','70대'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 13.72
P-value: 0.03
--------------------------
Expected Values
      treatment       control
10대  283.880753   9898.119247
20대  358.349963  12494.650037
30대  350.933711  12236.066289
40대  355.589779  12398.410221
50대  362.169612  12627.830388
60대  360.580415  12572.419585
70대   36.495768   1272.504232
--------------------------
Observed Values
     treatment  control
10대        284     9898
20대        361    12492
30대        306    12281
40대        337    12417
50대        409    12581
60대        372    12561
70대         39     1270
 


### 1. 나이에 따른 가입자수 상관관계 결과(차이가 있다)
1. 유의성검정 : P-value가 0.03로 0.05보다 작으므로 귀무가설을 기각한다(차이가 있다)
2. 기각역 : 검정통계량 13.72는 기각역(자유도 = 6)12.59 보다 크므로 귀무가설 기각역에 포함된다(차이가 있다)

### 2.성별에 따른 가입자수 상관관계 구하기

In [102]:
treatment.head(1)

,user_id,age,gender,account_state,account_city,account_country,latitude,longitude,account_traffic_source,account_created_at,order_id,order_items_id,status,sale_price,order_created_at,session_id,sequence_number,login_user_id,sess_created_at,sess_traffic_source,uri,event_type,ip_address,sess_state,sess_city,inventory_item_id,product_id,category,brand,product_name,inventory_created_at,inventory_sold_at,cost,age2
211,4108.0,62.0,M,Hunan,Zhumadian,China,29.081684,111.694935,Search,2024-01-12,5186.0,7479.0,Complete,21.799999,2024-01-15,a58138f5-1b41-4a5a-866a-97789e4f06e6,13.0,4108.0,2024-01-15,Adwords,/purchase,purchase,99.137.110.60,Hunan,Zhumadian,20200.0,17038.0,Tops & Tees,WinnieFashion,Â Â Exclusive Hawaiian Tropic Sunset In Paradi...,2023-05-06,NaT,12.1862,60


In [192]:
# 실험군의 성별 가입자수
test_F = treatment[treatment['gender']=='M']['user_id'].nunique()
test_M = treatment[treatment['gender']=='F']['user_id'].nunique()
# 대조군의 성별 가입자수
cont_F = control[control['gender']=='M']['user_id'].nunique()
cont_M = control[control['gender']=='F']['user_id'].nunique()

## Part1. Chi-square Test Report
genderF = [test_F, cont_F] # A와 B의 여성 유저 수
genderM = [test_M, cont_M] # A와 B의 남성 유저 수

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([genderF,genderM], columns=['treatment', 'control'], index=['F','M'])

In [193]:
cont_table

,treatment,control
F,1057,36640
M,1051,36860


In [194]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([genderF,genderM])

In [195]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['F','M'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 0.06
P-value: 0.81
--------------------------
Expected Values
     treatment       control
F  1051.016771  36645.983229
M  1056.983229  36854.016771
--------------------------
Observed Values
   treatment  control
F       1057    36640
M       1051    36860
 


### 2. 성별에 따른 가입자수 상관관계 결과(차이가 없다)
1. 유의성검정 : P-value가 0.81로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)
2. 기각역 : 검종통계량 0.06은 기각역(3.838)보다 작으므로 귀무가설 채택역에 포함된다


### 3. 국가에 따른 가입자수 상관관계

In [125]:
treatment['account_country'].value_counts()

account_country
China             1614
United States     1093
Brasil             739
South Korea        212
United Kingdom     205
Germany            187
France             165
Spain              153
Japan              112
Australia           85
Belgium             68
Poland               5
Colombia             1
Name: count, dtype: int64

In [198]:
# 실험군의 국가별 가입자수
test_China = treatment[treatment['account_country']=='China']['user_id'].nunique()
test_United_States = treatment[treatment['account_country']=='United States']['user_id'].nunique()
test_Brasil =treatment[treatment['account_country']=='Brasil']['user_id'].nunique()
test_South_Korea =treatment[treatment['account_country']=='South Korea']['user_id'].nunique()
test_United_Kingdom = treatment[treatment['account_country']=='United Kingdom']['user_id'].nunique()
test_Germany = treatment[treatment['account_country']=='Germany']['user_id'].nunique()
test_France =treatment[treatment['account_country']=='France']['user_id'].nunique()
test_Spain = treatment[treatment['account_country']=='Spain']['user_id'].nunique()
test_Japan = treatment[treatment['account_country']=='Japan']['user_id'].nunique()
test_Australia = treatment[treatment['account_country']=='Australia']['user_id'].nunique()
test_Belgium = treatment[treatment['account_country']=='Poland']['user_id'].nunique()
test_Poland  = treatment[treatment['account_country']=='Spain']['user_id'].nunique()
test_Colombia  = treatment[treatment['account_country']=='Colombia']['user_id'].nunique()
# 대조군의 국가별 가입자수
cont_China = control[control['account_country']=='China']['user_id'].nunique()
cont_United_States = control[control['account_country']=='United States']['user_id'].nunique()
cont_Brasil = control[control['account_country']=='Brasil']['user_id'].nunique()
cont_South_Korea = control[control['account_country']=='South Korea']['user_id'].nunique()
cont_United_Kingdom = control[control['account_country']=='United Kingdom']['user_id'].nunique()
cont_Germany = control[control['account_country']=='Germany']['user_id'].nunique()
cont_France = control[control['account_country']=='France']['user_id'].nunique()
cont_Spain = control[control['account_country']=='Spain']['user_id'].nunique()
cont_Japan = control[control['account_country']=='Japan']['user_id'].nunique()
cont_Australia =control[control['account_country']=='Australia']['user_id'].nunique()
cont_Belgium =control[control['account_country']=='Poland']['user_id'].nunique()
cont_Poland  = control[control['account_country']=='Spain']['user_id'].nunique()
cont_Colombia  = control[control['account_country']=='Colombia']['user_id'].nunique()

## Part1. Chi-square Test Report
China  = [test_China,cont_China]  
United_States  = [test_United_States ,cont_United_States ]  
Brasil  = [test_Brasil ,cont_Brasil ]  
South_Korea = [test_South_Korea,cont_South_Korea]   
United_Kingdom = [test_United_Kingdom,cont_United_Kingdom]   
Germany  = [test_Germany,cont_Germany]  
France  = [test_France ,cont_France ]  
Spain  = [test_Spain,cont_Spain]  
Japan    = [test_Japan ,cont_Japan ]      
Australia   = [test_Australia,cont_Australia]  
Belgium  = [test_Belgium,cont_Belgium]  
Poland  = [test_Poland,cont_Poland]  
Colombia  = [test_Colombia,cont_Colombia]  

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ], columns=['treatment', 'control'], index=['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])

In [199]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([China,United_States,Brasil ,South_Korea    ,United_Kingdom , Germany   ,
France  ,Spain,Japan  ,Australia  ,Belgium,Poland   , Colombia  ])

In [200]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ['China', 'United States', 'Brasil', 'South Korea', 'United Kingdom',
       'Germany', 'France', 'Spain', 'Japan', 'Australia', 'Belgium', 'Poland',
       'Colombia'])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 11.48
P-value: 0.49
--------------------------
Expected Values
                 treatment       control
China           717.858343  25096.141657
United States   474.224774  16578.775226
Brasil          304.284729  10637.715271
South Korea     111.986346   3915.013654
United Kingdom   96.691464   3380.308536
Germany          86.902740   3038.097260
France           98.109717   3429.890283
Spain            84.705838   2961.294162
Japan            51.752320   1809.247680
Australia        44.605438   1559.394562
Belgium           4.838745    169.161255
Poland           84.705838   2961.294162
Colombia          0.333707     11.666293
--------------------------
Observed Values
                treatment  control
China                 723    25091
United States         461    16592
Brasil                335    10607
South Korea           103     3924
United Kingdom         80     3397
Germany                84     3041
France                 9

### 3. 나라에 따른 가입자수 상관관계 결과 (차이가 없다)
1. 유의성검정 : P-value가 0.49로 0.05보다 크므로 귀무가설 채택
2. 가설검정 : 카이제곱의 값11.4 으로 기각역(자유도12)21.03보다 작으므로 채택한다


### 4. 유입경로에 따른 가입자수 상관관계

In [201]:
df['sess_traffic_source'].unique()

array([nan, 'Organic', 'YouTube', 'Email', 'Adwords', 'Facebook'],
      dtype=object)

In [203]:
# 실험군의 유입경로 가입자수
test_Email  = treatment[treatment['sess_traffic_source']=='Email']['user_id'].nunique()
test_Adwords  = treatment[treatment['sess_traffic_source']=='Adwords']['user_id'].nunique()
test_Facebook  =treatment[treatment['sess_traffic_source']=='Facebook']['user_id'].nunique()
test_YouTube  =treatment[treatment['sess_traffic_source']=='YouTube']['user_id'].nunique()
test_Organic  = treatment[treatment['sess_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control[control['sess_traffic_source']=='Email']['user_id'].nunique()
cont_Adwords  = control[control['sess_traffic_source']=='Adwords']['user_id'].nunique()
cont_Facebook = control[control['sess_traffic_source']=='Facebook']['user_id'].nunique()
cont_YouTube = control[control['sess_traffic_source']=='YouTube']['user_id'].nunique()
cont_Organic = control[control['sess_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Adwords =[test_Adwords, cont_Adwords]
Facebook =[test_Facebook, cont_Facebook]
YouTube =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Adwords ,Facebook,YouTube ,Organic ], columns=['treatment', 'control'],index=["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])

In [204]:
## Part3. 카이제곱 독립성 검정 모델 선언
chi2, p_val, d_f, expected = chi2_contingency([Email ,Adwords ,Facebook,YouTube ,Organic ])

In [205]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Adwords" ,"Facebook","YouTube" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 2.33
P-value: 0.68
--------------------------
Expected Values
           treatment       control
Email     938.051372  29643.948628
Adwords   667.359352  21089.640648
Facebook  244.036908   7711.963092
YouTube   241.092269   7618.907731
Organic   123.460100   3901.539900
--------------------------
Observed Values
          treatment  control
Email           913    29669
Adwords         683    21074
Facebook        247     7709
YouTube         236     7624
Organic         135     3890
 


### 4. 유입경로에 따른 가입자수 상관관계 결과 -> 차이없다
1. 유의성검정 : P-value가 0.68로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)
2. 기각역 : 검정통계량 2.33은 가격역(자유도3) 7.815보다 작으므로 채택역에 속하여 귀무가설을 채탁한다

### 5. 가입경로에 따른 가입자수 상관관계 

In [181]:
df['account_traffic_source'].unique()

array(['Search', nan, 'Email', 'Display', 'Facebook', 'Organic'],
      dtype=object)

In [206]:
# 실험군의 유입경로 가입자수
test_Email  = treatment[treatment['account_traffic_source']=='Email']['user_id'].nunique()
test_Display  = treatment[treatment['account_traffic_source']=='Display']['user_id'].nunique()
test_Facebook  =treatment[treatment['account_traffic_source']=='Facebook']['user_id'].nunique()
test_Search  =treatment[treatment['account_traffic_source']=='Search']['user_id'].nunique()
test_Organic  = treatment[treatment['account_traffic_source']=='Organic']['user_id'].nunique()

# 대조군의 유입경로 가입자수
cont_Email = control[control['account_traffic_source']=='Email']['user_id'].nunique()
cont_Display  = control[control['account_traffic_source']=='Display']['user_id'].nunique()
cont_Facebook = control[control['account_traffic_source']=='Facebook']['user_id'].nunique()
cont_Search = control[control['account_traffic_source']=='Search']['user_id'].nunique()
cont_Organic = control[control['account_traffic_source']=='Organic']['user_id'].nunique()


## Part1. Chi-square Test Report
Email = [test_Email, cont_Email]
Display =[test_Display, cont_Display]
Facebook =[test_Facebook, cont_Facebook]
Search =[test_YouTube, cont_YouTube]
Organic =[test_Organic, cont_Organic]

## Part2. Contingency Table & Chi-squaure Model
## 기초 테이블 형성
cont_table = pd.DataFrame([Email ,Display ,Facebook,Search ,Organic ], columns=['treatment', 'control'], index=["Email" ,"Display" ,"Facebook","Search" ,"Organic"])

In [207]:
## 기대값 표 형성
ex = pd.DataFrame(expected, columns = ['treatment', 'control'], index = ["Email" ,"Display" ,"Facebook","Search" ,"Organic"])
 
print(  '  '
 , '[Chi-square Analysis Result Report]'
 , 'Chi-square: {}'.format(round(chi2, 2))
 , 'P-value: {}'.format(round(p_val, 2))
 , '--------------------------'
 , 'Expected Values'
 , ex
 , '--------------------------'
 , 'Observed Values'
 , cont_table
 , '=========================='
 , ' '
 
 , sep = '\n')

  
[Chi-square Analysis Result Report]
Chi-square: 2.33
P-value: 0.68
--------------------------
Expected Values
           treatment       control
Email     938.051372  29643.948628
Display   667.359352  21089.640648
Facebook  244.036908   7711.963092
Search    241.092269   7618.907731
Organic   123.460100   3901.539900
--------------------------
Observed Values
          treatment  control
Email            85     3622
Display          84     3017
Facebook        131     4241
Search          236     7624
Organic         298    11123
 


### 5. 가입경로에 따른 가입자수 상관관계 결과 -> 차이없다
1. 유의성검정 : P-value가 0.68로 0.05보다 크므로 귀무가설을 채택한다(차이가 없다)
2. 기각역 : 검정통계량 2.33은 가격역(자유도3) 7.815보다 작으므로 채택역에 속하여 귀무가설을 채탁한다